In [1]:
import sys
sys.path.insert(0, '../../')

import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import schedule as sch
import topo
import typing
import utils
import yaml
from tqdm.notebook import tqdm

matplotlib.rc('font', family='Times New Roman', size=16)

In [2]:
def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

def gen_graphs(graph_count, source_selector_dict):
    source_selector = graph.MultiDomainSourceSelector(source_selector_dict)
    gen_args_list = [
            {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 3),
            "max_predecessors": random.randint(1, 2),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
            "sources_num": random.randint(1, 3),
        }
        for _ in range(graph_count)
    ]
    return [
        graph.MultiSourceGraphGenerator("g" + str(idx), **gen_args).gen_dag_graph()
        for idx, gen_args in enumerate(gen_args_list)
    ]

In [3]:
# 多边缘的流式计算图生成测试
graph_list = gen_graphs(3, {"edge0": {'e0rasp1': 8, 'e0rasp2': 8, 'e0rasp3': 8},
                        "edge1": {'e1rasp1': 8, 'e1rasp2': 8, 'e1rasp3': 8}})
for graph in graph_list:
    print(graph.uuid, ":")
    print(graph.g.nodes.data("color"))
    print(graph.g.nodes.data("type"))
    print(graph.g.edges)
    print([n.uuid for n in graph.get_cloud_vertices()])

g0 :
[('g0-v0', 'edge1'), ('g0-v1', 'edge1'), ('g0-v2', 'edge1'), ('g0-v3', 'cloud'), ('g0-v4', 'edge1'), ('g0-v5', 'cloud'), ('g0-v6', 'cloud'), ('g0-v7', 'cloud'), ('g0-v8', 'cloud'), ('g0-v9', 'cloud')]
[('g0-v0', 'source'), ('g0-v1', 'source'), ('g0-v2', 'operator'), ('g0-v3', 'sink'), ('g0-v4', 'operator'), ('g0-v5', 'sink'), ('g0-v6', 'sink'), ('g0-v7', 'sink'), ('g0-v8', 'sink'), ('g0-v9', 'sink')]
[('g0-v0', 'g0-v2'), ('g0-v0', 'g0-v9'), ('g0-v1', 'g0-v4'), ('g0-v2', 'g0-v3'), ('g0-v2', 'g0-v8'), ('g0-v4', 'g0-v5'), ('g0-v4', 'g0-v6'), ('g0-v4', 'g0-v7')]
['g0-v3', 'g0-v5', 'g0-v6', 'g0-v7', 'g0-v8', 'g0-v9']
g1 :
[('g1-v0', 'edge1'), ('g1-v1', 'edge0'), ('g1-v2', 'edge0'), ('g1-v3', 'edge0'), ('g1-v4', 'edge0'), ('g1-v5', 'cloud'), ('g1-v6', 'cloud')]
[('g1-v0', 'source'), ('g1-v1', 'source'), ('g1-v2', 'operator'), ('g1-v3', 'operator'), ('g1-v4', 'operator'), ('g1-v5', 'sink'), ('g1-v6', 'sink')]
[('g1-v0', 'g1-v6'), ('g1-v1', 'g1-v2'), ('g1-v2', 'g1-v3'), ('g1-v3', 'g1-v4')

In [4]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e6h.yaml", "r").read(), Loader=yaml.Loader))
print(sc.topo.g.nodes)

['cloud0_router', 'cloud_switch', 'cloud1', 'edge0_router', 'e0rasp_switch', 'e0rasp1', 'e0rasp2', 'e0rasp3', 'edge1_router', 'e1rasp_switch', 'e1rasp1', 'e1rasp2', 'e1rasp3']


In [6]:
sc.topo.clear_occupied()
flow_scheduler = sch.MultiEdgeFlowScheduler(sc)
flow_calculator = sch.LatencyCalculator(sc.topo)
flow_result_list = flow_scheduler.schedule_multiple(graph_list)
for g, result in zip(graph_list, flow_result_list):
    assert result is not None
    flow_calculator.add_scheduled_graph(g, result)
    print()
flow_latency, flow_bp, _ = flow_calculator.compute_latency()
print(flow_latency)




{'g0': 31.797861811391225, 'g1': 31.97833333333333, 'g2': 27.924866310160425}
